# Mortgage Calculator with one-off repayments

Based on [https://onladder.co.uk/blog/how-to-calculate-mortgage-repayments/](https://onladder.co.uk/blog/how-to-calculate-mortgage-repayments/)

Especially in the UK, you can make overpayments totalling up to 10\% of your original loan amount (not the current loan) during a 12 month period without incurring any applicable early repayment charges.

For instance, let's assume you have received a bonus payment from your employer (e.g., £1,000), and you decide to make a one-off overpayment to increase you equity against your mortgage. This code helps you understanding how the interest (cost of borrowing) changes when we perform such one-off payments.

## Parameters

All the numbers you need to get started

In [1]:
# Principal (starting balance) of the loan
mortgage_amount = 200000

# Annual interest rate (APRC)
# Can be expressed as either in percentage (e.g., 3.99%) or decimal (e.g., 0.0399)
interest_rate = 0.0399 

# Duration of your mortgage in years
mortgage_period = 30 

# Number of payments in total: if you make one mortgage payment every month for 25 years, that’s 25*12 = 300
# Duration of your mortgage in months
total_instalments = mortgage_period*12

# For displaying purposes
currency = "£"# "$" "€"

## One-off payments
These are set of payments that occur less frequently, differenly from recurring payments.

In [9]:
# structure [(month, amount), (month, amount)]
repayments = [(2,5000), # giving £5,000 at the second month
            (15,1000)] # giving £1,000 at the 15th month

## Imports

In [3]:
import pandas as pd
from miscellaneous import *
from IPython.display import HTML, IFrame, display

## Original Summary
Without repayments

In [4]:
monthly_payment = payments(mortgage_amount,interest_rate,mortgage_period)
total_given = approx(monthly_payment*mortgage_period*12) 
print(f"Total amount borrowed {currency}{clean(mortgage_amount)}, with and interest rate of {clean(denormalise_interest_rate(interest_rate))}, and a repayment over {mortgage_period} ({mortgage_period*12} instalments)")
print(f"Montly payments set to {currency}{clean(monthly_payment)}")
print(f"Total payments {currency}{clean(total_given)}")
print(f"For each £1 borrowed you are will pay back {currency}{clean(total_given/mortgage_amount)}")

Total amount borrowed £200,000.00, with and interest rate of 3.99, and a repayment over 30 (360 instalments)
Montly payments set to £953.68
Total payments £343,324.80
For each £1 borrowed you are will pay back £1.72


## Payment Schedule

In [5]:
repayments_adj = {rep[0]:rep[1] for rep in repayments}

columns=['Principal to date','Payment','One-off','Paid to date','Interest charged', 'Interest charged to date', 'Principal repaid', 'Principal repaid to date', 'Remaining principal']
table = pd.DataFrame(columns=columns, index=[x for x in range(1, total_instalments+1)])
remaining_principal = mortgage_amount
payment_to_date = 0
interest_paid_to_date = 0
principal_repaid_to_date = 0
to_break = False
for instalment in range(1, total_instalments+1):
    one_off_repayment = False
    this_month_payment = monthly_payment
    if instalment in repayments_adj:
        one_off_repayment = True
        this_month_payment += repayments_adj[instalment]
    
    principal_to_date = remaining_principal  
    curr_interest_paid = current_interest_paid(principal_to_date, interest_rate)
    if principal_to_date <= this_month_payment:
        this_month_payment = principal_to_date + curr_interest_paid
        to_break = True
    
    payment_to_date += this_month_payment
    
    interest_paid_to_date += curr_interest_paid
    principal_repaid = this_month_payment - curr_interest_paid
    principal_repaid_to_date += principal_repaid
    remaining_principal -= principal_repaid
    table.loc[instalment] = pd.Series({columns[0]:clean(principal_to_date),
                                         columns[1]:clean(this_month_payment),
                                         columns[2]:"Y" if one_off_repayment else "",
                                         columns[3]:clean(payment_to_date),
                                         columns[4]:clean(curr_interest_paid),
                                         columns[5]:clean(interest_paid_to_date),
                                         columns[6]:clean(principal_repaid),
                                         columns[7]:clean(principal_repaid_to_date),
                                         columns[8]:clean(remaining_principal)})
    if to_break:
        table = table.drop(range(instalment+1, total_instalments+1))
        break

### Details
- **Principal to date**:  This is the amount of the loan at a given time.

- **Payment**: This refers to the total amount of money paid toward the loan in a given period (usually monthly). This payment typically includes both principal and interest.

- **Paid to date**:  The total amount of money paid towards the loan since it originated. This includes both principal and interest portions of all payments made.

- **One-off**: Whether for this month there was a one-off repayment.

- **Interest charged**: The amount of interest that accrues on the loan for a specific period (e.g., a month). This is the cost of borrowing the money.

- **Interest charged to date**: The total amount of interest that has accrued on the loan since it originated.

- **Principal repaid**: The portion of a specific payment that goes towards reducing the original loan amount (the principal).

- **Principal repaid to date**: The total amount of the original loan amount that has been paid back since the loan originated.

- **Remaining principal**: The outstanding balance on the loan; this is the amount still owed. It's calculated as "Principal to date" minus "Principal repaid to date".

In [6]:
table

,Principal to date,Payment,One-off,Paid to date,Interest charged,Interest charged to date,Principal repaid,Principal repaid to date,Remaining principal
1,"200,000.00",953.68,,953.68,665.00,665.00,288.68,288.68,"199,711.32"
2,"199,711.32","5,953.68",Y,"6,907.36",664.04,"1,329.04","5,289.64","5,578.32","194,421.68"
3,"194,421.68",953.68,,"7,861.04",646.45,"1,975.49",307.23,"5,885.55","194,114.45"
4,"194,114.45",953.68,,"8,814.72",645.43,"2,620.92",308.25,"6,193.80","193,806.20"
5,"193,806.20",953.68,,"9,768.40",644.41,"3,265.33",309.27,"6,503.07","193,496.93"
...,...,...,...,...,...,...,...,...,...
337,"3,907.86",953.68,,"327,390.16",12.99,"130,357.33",940.69,"197,032.83","2,967.17"
338,"2,967.17",953.68,,"328,343.84",9.87,"130,367.20",943.81,"197,976.64","2,023.36"
339,"2,023.36",953.68,,"329,297.52",6.73,"130,373.93",946.95,"198,923.59","1,076.41"
340,"1,076.41",953.68,,"330,251.20",3.58,"130,377.51",950.10,"199,873.69",126.31


## New Summary

In [7]:
print(f"Total amount borrowed {currency}{clean(mortgage_amount)}, with an initial interest rate of {clean(denormalise_interest_rate(interest_rate))}.")
print(f"Thanks to early repayments you shortened your mortgage to {instalment} instalments")
print(f"Montly payments was initially set to {currency}{clean(monthly_payment)}")
print(f"Then you performed {len(repayments)} one-off repayments, of a total of {currency}{clean(sum([rep[1] for rep in repayments]))}")
print(f"Your total repayment is {currency}{clean(payment_to_date)} (instead of the original {currency}{clean(total_given)})")
print(f"For each £1 borrowed you are will pay back {currency}{clean(payment_to_date/mortgage_amount)}")

Total amount borrowed £200,000.00, with an initial interest rate of 3.99.
Thanks to early repayments you shortened your mortgage to 341 instalments
Montly payments was initially set to £953.68
Then you performed 2 one-off repayments, of a total of £6,000.00
Your total repayment is £330,377.93 (instead of the original £343,324.80)
For each £1 borrowed you are will pay back £1.65


## Export to Excel

In [8]:
output_file = "my_mortgage_analysis2.xlsx"
table.to_excel(output_file)